In [1]:
from torchvision.transforms import transforms #библиотека для работы с изображениями
import torch

In [2]:
#open CV
##import cv2

##image = cv2.imread('pok.png')
##image.shape

#возвращает np.array
#открывает картинку в BGR (обратно RGB)
#сначала ширина высота, потом каналы

In [3]:
from torchvision.io import read_image
#сначала количество каналов, а потом ширина - высота

image = read_image('pok.png')
image.shape


#возвращает tensor

torch.Size([4, 2206, 1938])

In [4]:
f_t = transforms.Lambda(lambda x: x*2) #Объект занесенный в функцию должен быть возведен в квадрат
a = torch.rand(4)
f_t(a)
#transforms.  -- посмотреть какие есть функции. Большинство функций работают с изображениями формата PIL (ToPILImage()) - поэтому надо менять форматы
#compose - п

tensor([1.7025, 1.0056, 1.2843, 0.8057])

In [5]:
f_t1 = transforms.Compose([f_t, f_t]) #объединяет функции в одну функцию, включая функции из transform.
f_t1(a)

tensor([3.4049, 2.0112, 2.5686, 1.6114])

In [6]:
#Если датасет слишком маленький, можно
#Изменять изображения - переворачивать, добавлять блюр, менять цветовую гамму
#2 Способ - взять натренированную модель. Взять слои которые натренированны на поиске низкоуровневых паттернов. И переобучаем модель на что-то новое

In [7]:
from torch.utils.data import Dataset
import pathlib 
#dataset - откуда забрать 1 объект
#dataloader - собирает батчи

In [8]:
#Класс того, как будет выдаваться один объект

class LatinDataset(Dataset):

    #в ините объявляем путь к датасету
    def __init__(self, path_dataset: pathlib.Path):
        self.path_dataset = path_dataset
        self.data_list = [x for x in self.path_dataset.glob('**/*') if x.is_file()] # **/* - паттерн значащий - любая папка внутри и не важно что после неё
        # то есть передаем в класс условие по которому ходить по датасету
        # ------------
        # Выделяем количество классов:
        self.data_class = list(set([x.parent for x in self.data_list])) #Используем множество т.к. там значения уникальны. Т.е. убираем повторяющиеся пути
        # Возвращаем в лист, т.к. в сете нет индексов, а они нам нужны
        # Для свертки важно лишь количество фильтров, а не размерность картинки.
        # Но для батчей все картинки должны быть одной размерности. Поэтому все картинки нужно подогнать к одному размеру.
        self.transform_func = transforms.Compose([
             transforms.ToPILImage(),
             transforms.Grayscale(), #т.к. мы работаем с чб картинками| 1 фильтр
             transforms.Resize((124, 124)),
             transforms.ToTensor()
        ])

    def __len__(self): #всегда определять длину. Т.к. обращаемся к датасету он должен возвращать максимальное кол-во объектов в датасете
        return len(self.data_list)
    
    def __getitem__(self, index): #dataloader returns some index from datalist
        img_path = self.data_list[index] #path to the image
        #but path also returns label
        #label вектор размерности максимального размера класса, где единица указывает принадлежность к классу (см. one hot encoder)
        img_label = torch.zeros(len(self.data_class)) #возвращает тензор с нулевыми значениями с размерностью кол-ва классов
        img_label[self.data_class.index(img_path.parent)] = 1.0 #Находим индекс класса и заменяем ячейку под этим индексом на принадлежность к классу.
        #В нашем случае каждая папка это отдельный класс
        img_tensor = read_image(str(img_path)) #Нужно str т.к. read_image не работает с типом WindowsPath, только str
        return self.transform_func(img_tensor), img_label
    

In [9]:
# g = pathlib.Path(r'C:\Users\Joe\Desktop\python_yes\what_is_love\Latin')
# a = [x for x in g.glob('**/*') if x.is_file()]
# set([x.parent for x in a])
# #[a for a in g.iterdir()]

Pytorch это динамически генерируемы граф <br>
Поэтому нужно инициализировать все начальные слои, а потом описать взаимодействие функций в классе

In [10]:
import torch.nn as nn #Модули для построения нейронных сетей. Обычно торч работает с тензорами и подобным
# nn - модель нейросети. Но также и слой (наприм. слой с функцией активации)

In [11]:
class CVModel(nn.Module):

    def __init__(self):
        super(CVModel, self).__init__() # Initialize self._modules as OrderedDict
        self.conv_1 = nn.Conv2d(1, 32, (3, 3)) #Объявляем свёрточный слой. Ему необходимо несколько параметров: кол-во слоев, кол-во фильтров, размер фильтра (окна которым ходим по картинке)
        # 64 фильтра(паттерна) на цветной картинке стоит
        self.conv_2 = nn.Conv2d(32, 64, (3, 3)) #Высокоуровневые паттерны (кол-во фильтров обычно увеличивают в 2-3 раза)
        self.pool_1 = nn.MaxPool2d((2, 2)) #Уменьшаем входящую картинку на 2
        # Нужно всегда просчитывать размер картинки на фыходе, чтобы не искать свертку 1 пикселя
        # Если картинка размера (n, n), окна (m, m) и шаг окна = 1. То на выходе изображение будер размера (n-m+1, n-m+1)
        self.conv_3 = nn.Conv2d(64, 128, (3, 3))
        self.conv_4 = nn.Conv2d(128, 128, (3, 3))
        self.pool_2 = nn. MaxPool2d((2, 2))
        self.glob_pool = nn.MaxPool2d((28, 28)) #Растянули матрицу на вектор
        self.linear = nn.Linear(128, 26) #26 классов - кол-во букв в алфавите  #Linear + Softmax ~ многоклассовая логистическая классификация
        self.softmax = nn.Softmax()
# (124, 124, 1) -> (122, 122, 32) -> (120, 120, 64) -> (60, 60, 64) -> (58, 58, 128) -> (56, 56, 128) -> (28, 28, 128) -> (1, 1, 128)
#                  32 матрицы размером 122х122          pooling
# Linear - создает 26 ячеек в каждой из которых проводится линейная операция на вектором размерностью 128. Инициализируем 26 линейных регрессий. Каждая регрессия определяет принадлежит ли объет к классу или нет (булиевая классификация)

    def forward(self, X):
        res = self.conv_1(X)
        res = self.conv_2(res)
        res = self.pool_1(res)
        res = self.conv_3(res)
        res = self.conv_4(res)
        res = self.pool_2(res)
        res = self.glob_pool(res) #(1, 1, 128)
        res = self.linear(res.view(-1, 128)) #Сокращение размерности до (128)
        return self.softmax(res)
# loss should be CrossEntropy

In [12]:

from torch.utils.data import DataLoader

path = pathlib.Path(r'C:\Users\Joe\Desktop\python_yes\what_is_love\Latin')
loss = nn.CrossEntropyLoss()
dataset = LatinDataset(path)
model = CVModel()
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.RMSprop(params=model.parameters())

In [13]:
def train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset):
    correct = 0
    epoch_loss = 0.0 
    for i, data in enumerate(trainloader, 0):
        x, y = data
        optimizer.zero_grad()
        outputs = model(x)
        error = loss(outputs, y)
        error.backward()
        epoch_loss += outputs.shape[0] * error.item()
        optimizer.step()
        outputs = (outputs > 0.5).float()
        correct += (outputs == y).float().sum()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss/ dataset.__len__():.3f}')

In [14]:
for epoch in range(1):
    train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset)

c:\Users\Joe\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


TRAIN [1] loss: 3.283


In [15]:
# def train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset):
#     #кол-во эпох, оптимизатор - из старых весов отнимаем значения градиента и получаем новое
#     correct = 0
#     epoch_loss = 0.0 #для себя чтобы следить за изменение й ошибки
#     for i, data in enumerate(trainloader, 0):
#         x, y = data #так как датасет возвращает 2 значения
#         optimizer.zero_grad()
#         outputs = model(x) #результаты модели
#         loss = loss(outputs, y) #функция ошибки
#         loss.backward() #обратные градиенты (т.е считает призводную кажд элемента)
#         epoch_loss += outputs.shape[0] * loss.item() #записываем значения ошибок
#         optimizer.step() #совершает ход вычитания производных и старых w
#         outputs = (outputs > 0.5).float() #для себя
#         correct += (outputs == y).float().sum()

#     print(f'TRAIN [{epoch + 1}] loss: {epoch_loss/ dataset.__len__():.3f}')